In [1]:
import pandas as pd

In [2]:
data_path = "C:/Users/Katta's/PycharmProjects/data/usecase3/train.csv"
housing = pd.read_csv(data_path)

In [3]:
 house_obj= housing.select_dtypes(include=['object']).copy()

In [4]:
house_obj.drop(house_obj.columns[house_obj.isnull().any()],axis=1, inplace=True) 

In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in house_obj.columns.values:
        le.fit(house_obj[col])
        house_obj[col]=le.transform(house_obj[col])

In [7]:
# All types except object
housing_int = housing.select_dtypes(exclude=['object']).copy()
# housing_cat1 = housing_cat.astype('int64',copy=False)
int_columns = housing_int.columns


In [8]:
from sklearn.preprocessing import StandardScaler
# Scaling training because of variance
ss = StandardScaler()
# scaling training data
housing_cat1_scaled = ss.fit_transform(housing_int)


c:\users\katta's\venv\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\katta's\venv\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [9]:
housing_cat1_scaled_df = pd.DataFrame(housing_cat1_scaled, columns=int_columns)

In [10]:
housing_t = pd.concat([house_obj, housing_cat1_scaled_df],axis= 1)


In [11]:
housing_t['GarageYrBlt'].fillna(housing_t['GarageYrBlt'].mean(), inplace=True)
housing_t['MasVnrArea'].fillna(housing_t['MasVnrArea'].mean(), inplace=True)
housing_t['LotFrontage'].fillna(housing_t['LotFrontage'].mean(), inplace=True)

In [12]:
housing_train = housing_t.dropna()
print(housing_train.columns)

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive',
       'SaleType', 'SaleCondition', 'Id', 'MSSubClass', 'LotFrontage',
       'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')


In [13]:
train_x = housing_train.drop(['SalePrice', 'Id'],axis=1)
train_y = housing_train['SalePrice']
train_x.columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive',
       'SaleType', 'SaleCondition', 'MSSubClass', 'LotFrontage', 'LotArea',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [14]:
# Early stopping 
from keras.callbacks import EarlyStopping
_es_monitor = 'val_loss'
_es_patience = 10
es = EarlyStopping(monitor=_es_monitor, mode='min', verbose=1, patience=_es_patience)

Using TensorFlow backend.


In [15]:
#define batch size and number of epchos 
_batch_size = 32
_epochs = 500
_mc_model_location = 'v1_model.h5'
_mc_monitor = 'val_loss'
lr = 0.005
# Defining optimizer 
from keras.optimizers import SGD
_opt = SGD(lr)
_loss = 'mean_squared_error'

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
# Model check point
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Dense(64, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='relu'))
mc = ModelCheckpoint(_mc_model_location, monitor=_mc_monitor, mode='max', verbose=1, save_best_only=True)

In [17]:
model.compile(loss=_loss, optimizer=_opt, metrics=['accuracy'])


In [20]:
# fix random seed for reproducibility
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=model, epochs=100, batch_size=5, verbose=0)


In [24]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, train_x, train_y, cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

TypeError: __call__() missing 1 required positional argument: 'inputs'

In [1]:
# plot loss - Train vs Valid
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='test loss')
plt.title('Loss - Train vs validation')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [87]:
metrics = model.evaluate(train_x, train_y)
print("\n%s: %.2f%%" % (model.metrics_names[0], metrics[0]*100))

  32/1460 [..............................] - ETA: 0s

1460/1460 [==============================] - 0s 27us/step



loss: 100.00%
